# AIML331 Assignment 3
### crowelenn, 300607096

In this assignment, we focus on using modern machine learning techniques to classify animals from the OxfordIIITPet dataset. The animals are to be classified into the following categories:

- long-haired cat
- short-haired cat
- long-haired dog
- short-haired dog

In the first part of the assignment, we will train and evaluate a convolutional neural network to perform this task.

#### Setup
Import libraries, and load the dataset. The function to load the dataset has been provided in `dataset_wrapper` and the data is stored in the `./data` local directory.

In [1]:
from torch import nn

from dataset_wrapper import get_pet_datasets

In [10]:
train_dataset, val_dataset, test_dataset = get_pet_datasets(img_width=128, img_height=128,root_path='./data' )

In [11]:
print(f"Loaded data, train = {len(train_dataset)}, test = {len(test_dataset)}")

Loaded data, train = 5719, test = 716


Import the required libraries:

In [16]:
import torch
import torchvision
import torch.nn as nn

In [8]:
# check that cuda is working
torch.cuda.is_available()

True

#### Dataloaders, training devices etc
All the pytorch stuff that isn't the neural network itself

In [19]:
# define the compute device
compute_device = torch.device('cuda:0')

In [21]:
# load the datasets
batch_size = 64
training_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

testing_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

#### Defining the neural network

In [17]:
class PetsConvNet(nn.Module):
    def __init__(self, num_classes=4):
        super(PetsConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3,
                      out_channels=16,
                      kernel_size=5,
                      stride=1,
                      padding=2),
            nn.BatchNorm2d(num_features=16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # l1 output dim = 128 - 5 + 1 + 2*2 / 2
        #               = 64
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=32,
                      kernel_size=5,
                      stride=1,
                      padding=2),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # l2 output dim = 64 - 5 + 1 + 2*2 / 2
        #               = 32

        # output = input - filter + 1 + 2*padding
        self.fc = nn.Linear(32*32*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

#### Building the model and training

In [23]:
model = PetsConvNet(num_classes=4).to(compute_device)

In [27]:
learning_rate = 0.01
num_epochs = 20

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

steps = len(training_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(training_dataloader):
        images = images.to(compute_device)
        labels = labels.to(compute_device)

        # forwards
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backpropogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, steps, loss.item()))

Epoch [1/20], Step [1/90], Loss: 0.9950
Epoch [1/20], Step [11/90], Loss: 1.2070
Epoch [1/20], Step [21/90], Loss: 0.9515
Epoch [1/20], Step [31/90], Loss: 1.0213
Epoch [1/20], Step [41/90], Loss: 1.1345
Epoch [1/20], Step [51/90], Loss: 1.1915
Epoch [1/20], Step [61/90], Loss: 0.9353
Epoch [1/20], Step [71/90], Loss: 1.1946
Epoch [1/20], Step [81/90], Loss: 1.1094
Epoch [2/20], Step [1/90], Loss: 0.9098
Epoch [2/20], Step [11/90], Loss: 0.9702
Epoch [2/20], Step [21/90], Loss: 0.9113
Epoch [2/20], Step [31/90], Loss: 1.0206
Epoch [2/20], Step [41/90], Loss: 0.9871
Epoch [2/20], Step [51/90], Loss: 1.0686
Epoch [2/20], Step [61/90], Loss: 1.0128
Epoch [2/20], Step [71/90], Loss: 1.0074
Epoch [2/20], Step [81/90], Loss: 1.0826
Epoch [3/20], Step [1/90], Loss: 0.9048
Epoch [3/20], Step [11/90], Loss: 0.8468
Epoch [3/20], Step [21/90], Loss: 0.8756
Epoch [3/20], Step [31/90], Loss: 1.0860
Epoch [3/20], Step [41/90], Loss: 1.0895
Epoch [3/20], Step [51/90], Loss: 1.2954
Epoch [3/20], Step 

In [32]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in validation_dataloader:
        images = images.to(compute_device)
        labels = labels.to(compute_device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the test images: 39.385474860335194 %
